# Chapter 2 - First Steps with LangChain



## ChatDeepSeek example

Taken from [langchain-deepseek](https://python.langchain.com/api_reference/deepseek/chat_models/langchain_deepseek.chat_models.ChatDeepSeek.html) documentation

In [1]:
import os
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv

load_dotenv() # load environment variables from .env file

# instantiate the llm
llm = ChatDeepSeek(model="deepseek-chat", api_key=os.environ.get("DEEPSEEK_API_KEY"))

# construct a prompt

messages = [
    ("system", "You are a helpful translator. Translate the user sentence to Spanish"),
    ("human", "I love programming.")
]

# pass the prompt to the llm and get a response
response = llm.invoke(messages)
print(response)

content='Me encanta programar.  \n\n*(Alternative options depending on context:)*  \n- **Amo programar.** (stronger emphasis)  \n- **Disfruto mucho programando.** ("I really enjoy programming")  \n\nLet me know if you\'d like a regional variation (e.g., Latin America vs. Spain) or a different tone!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 19, 'total_tokens': 90, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 19}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': 'db3b6cc4-4e7b-4ebf-853c-9b8ee4d94959', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--0fc05e59-ffb8-489c-b02f-323407d02631-0' usage_metadata={'input_tokens': 19, 'output_tokens': 71, 'total_tokens': 90, 'input_token_details': {'cache_read': 0}, 'output_token_details': {

## Development testing

During development, you might want to test your application without making an actual API calls. LangChain provides ```FakeListLLM``` for this purpose:

In [2]:
from langchain_community.llms import FakeListLLM

# create a fake LLM that always returns the same response

fake_llm = FakeListLLM(responses=["Hello There"])

response = fake_llm.invoke("Any input will return 'Hello There'")
print(response)

Hello There


## Working with chat models

In [3]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import SystemMessage, HumanMessage
from dotenv import load_dotenv
import os

load_dotenv() # load environment variables from .env file

# instantiate llm
chat = ChatDeepSeek(model="deepseek-chat", api_key=os.environ.get("DEEPSEEK_API_KEY"))

# construct prompt
messages = [
    SystemMessage(content="You're a helpful programming assistant."),
    HumanMessage(content="Write a Python function to calculate factorial."),
]

# invoke the llm
response = chat.invoke(messages)
print(response.content)

Here’s a Python function to calculate the factorial of a non-negative integer using both iterative and recursive approaches:

### 1. Iterative Approach:
```python
def factorial_iterative(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result
```

### 2. Recursive Approach:
```python
def factorial_recursive(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial_recursive(n - 1)
```

### Example Usage:
```python
num = 5
print(f"Iterative Factorial of {num}: {factorial_iterative(num)}")
print(f"Recursive Factorial of {num}: {factorial_recursive(num)}")
```

### Output:
```
Iterative Factorial of 5: 120
Recursive Factorial of 5: 120
```

### Notes:
- The factorial of `0` is `1` by definition.
- Negative numbers raise a `ValueError` since factorial is not

## Chat prompt templates

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
import os

load_dotenv()

# create a prompt template
template = ChatPromptTemplate.from_messages([
    ("system", "You're an English to Spanish translator."),
    ("human", "Translate this to Spanish: {text}.")
])

# instantiate llm
chat = ChatDeepSeek(model="deepseek-chat", api_key=os.environ.get("DEEPSEEK_API_KEY"))

# create a message from the template
formatted_messages = template.format(text="Hello, how are you?")

# invoke llm and get a response
response = chat.invoke(formatted_messages)
print(response.content)


La traducción al español es:  

**"Hola, ¿cómo estás?"**  

Si necesitas una versión más formal (por ejemplo, en un contexto profesional o con alguien a quien debes respeto), también puedes usar:  

**"Hola, ¿cómo está usted?"**  

¿Necesitas ayuda con algo más? 😊


## Simple workflows with LangChain Expression Language (LCEL)

In [4]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
import os

load_dotenv()

# create components
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
llm = ChatDeepSeek(model="deepseek-chat", api_key=os.environ.get("DEEPSEEK_API_KEY"))
output_parser = StrOutputParser()

# chain them together using LCEL
chain = prompt | llm | output_parser

# execute the workflow with a single call
response = chain.invoke({"topic": "programming"})
print(response)


Here's a classic programming joke for you:

**Why do programmers prefer dark mode?**  
Because light attracts bugs!  

*(And also because staring at a bright screen all day is painful... 😆)*  

Want another one? Just ask!


## Complex chain example

In [5]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatDeepSeek(model="deepseek-chat", api_key=os.environ.get("DEEPSEEK_API_KEY"))

# first chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
output_parser = StrOutputParser()
story_chain = story_prompt | llm | output_parser

# second chain analyses the story
analysis_prompt = PromptTemplate.from_template("Analyze the following story's mood:\n {story}")
analysis_chain = analysis_prompt | llm | output_parser

# combine the chains
story_with_analysis = story_chain | analysis_chain

# run the combined chain
response = story_with_analysis.invoke({"topic": "a rainy day"})
print(response)

The mood of **"The Rainy Day"** is **nostalgic, peaceful, and joyful**, with an underlying sense of warmth and childlike wonder.  

### **Key Elements of the Mood:**  
1. **Nostalgic & Reflective** – Clara’s love for rainy days and her spontaneous jump into a puddle evoke childhood memories, creating a wistful yet sweet tone.  
2. **Peaceful & Serene** – The description of the rain ("the world softened," "rhythmic patter") gives a soothing, almost meditative quality.  
3. **Joyful & Playful** – Clara’s laughter, Liam joining in, and their shared childlike behavior infuse the story with lighthearted happiness.  
4. **Hopeful & Warm** – The ending, where the sun reappears and Clara reflects on how rain brings out the best in people, leaves a comforting, optimistic impression.  

### **Supporting Details:**  
- **Imagery:** Soft rain, wet pavement, splashing puddles—all create a vivid, sensory-rich atmosphere.  
- **Dialogue & Actions:** Clara and Liam’s playful interaction reinforces the

While this works, we’ve lost the original story in our result – we only get the analysis! In production applications, we typically want to preserve context throughout the chain:

In [7]:
from langchain_core.runnables import RunnablePassthrough

# using RunnablePassthrough.assign to preserve data across the chain
enhanced_chain = RunnablePassthrough.assign(
    story=story_chain # add 'story' key with generated story
).assign(
    analysis=analysis_chain # add 'analysis' key with analysis of the story
)

# run the enhanced chain
response = enhanced_chain.invoke({"topic": "a rainy day"})
print(response.keys())
print(response)


dict_keys(['topic', 'story', 'analysis'])
{'topic': 'a rainy day', 'story': '**The Rainy Day**  \n\nThe sky had been gray all morning, heavy with the promise of rain. Clara sat by the window, watching the first drops splatter against the glass, blurring the world outside. The rhythmic patter of rain against the roof was soothing, like a lullaby.  \n\nShe pulled on her yellow raincoat and stepped outside, the cool mist kissing her cheeks. Puddles had already formed on the sidewalk, and she couldn’t resist jumping into the largest one, sending droplets flying.  \n\nA small, shivering kitten huddled under a nearby bench, its fur soaked. Clara knelt, offering her hand. The kitten hesitated, then pressed its tiny head against her palm. Wrapping it in her coat, she carried it home, where warmth and dry towels waited.  \n\nBy evening, the rain slowed to a drizzle. The kitten, now curled on Clara’s lap, purred softly as she sipped hot cocoa. Outside, the world glistened—fresh and new.  \n\nSom